In [1]:
!pip install rapidfuzz
from google.colab import drive

!pip install python-Levenshtein
import Levenshtein

drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 3.1 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
import pandas as pd
from scipy.spatial.distance import pdist
from rapidfuzz import process, fuzz
import numpy as np

In [3]:
# Get data, create usable format
movie_pivot = pd.read_csv('/content/drive/MyDrive/movie_pivot.csv', index_col=[0])
movie_meta = pd.read_csv('/content/drive/MyDrive/movie_meta.csv', index_col=[0])

movie_pivot = movie_pivot.loc[movie_pivot.index.intersection(movie_meta.index)]
movie_meta = movie_meta.loc[movie_pivot.index.intersection(movie_meta.index)]

# Create frame for storing distances
dist_frame = pd.DataFrame(index=movie_meta.index, data=movie_meta[['title']])

In [4]:
# Function to search movie name, returns index in database
def search(name, index):
    dist = []

    for t in movie_meta['title']:
        dist.append(Levenshtein.distance(t, name, weights=(100, 1, 10)))

    df = pd.DataFrame(movie_meta[['title']])
    df['dist'] = dist
    selection = df.sort_values('dist', ascending=True).head(10).reset_index()

    display(selection[['title']])

    print('\n\nSelected:')

    display(pd.DataFrame(selection.iloc[index]).T)

    return selection['item_id'][index]

In [11]:
# Search for movie 1. Store distances to other movies

base_id = search('Casion Royale', 0)
dist_frame['Dist_1'] = ((movie_pivot - movie_pivot.loc[base_id]) ** 2).sum(axis=1).to_list()

,title
0,Casino Royale (2006)
1,Casino Royale (1967)
2,Invasion of the Body Snatchers (1956)
3,Cats & Dogs: The Revenge of Kitty Galore (2010)
4,Invasion of the Body Snatchers (1978)
5,Washington Square (1997)
6,Confessions of a Shopaholic (2009)
7,"Confessional, The (Confessionnal, Le) (1995)"
8,Mission: Impossible (1996)
9,National Lampoon's Van Wilder (2002)




Selected:


,item_id,title,dist
0,49272,Casino Royale (2006),27


In [12]:
# Search for movie 2. Store distances to other movies

base_id = search('Airplane!', 1)
dist_frame['Dist_2'] = ((movie_pivot - movie_pivot.loc[base_id]) ** 2).sum(axis=1).to_list()

,title
0,Airplane! (1980)
1,Airplane II: The Sequel (1982)
2,Highlander (1986)
3,Compliance (2012)
4,Flightplan (2005)
5,"Angry Red Planet, The (1959)"
6,American Me (1992)
7,Purple Rain (1984)
8,Au revoir les enfants (1987)
9,Doppelganger (1993)




Selected:


,item_id,title,dist
1,2792,Airplane II: The Sequel (1982),31


In [15]:
# Store weighted average

w_1 = .5
w_2 = 1 - w_1

dist_frame['Dist_3'] = (dist_frame['Dist_1'] ** 2 * w_1) + (dist_frame['Dist_2'] ** 2 * w_2)

In [17]:
# Search based on one of the three types of distance (Dist_1, Dist_2, or Dist_3)

dist_frame.sort_values('Dist_1').head(20)

,title,Dist_1,Dist_2,Dist_3
item_id,,,,
49272,Casino Royale (2006),0.000000,818.927493,335321.119099
96079,Skyfall (2012),93.050414,787.071359,314069.851788
54286,"Bourne Ultimatum, The (2007)",141.867862,893.190644,408958.008812
2947,Goldfinger (1964),148.822576,733.209218,279871.958302
2949,Dr. No (1962),171.262008,709.577715,266415.604841
2948,From Russia with Love (1963),198.907784,690.651674,258282.020547
63113,Quantum of Solace (2008),202.612840,639.216335,224824.743136
5418,"Bourne Identity, The (2002)",211.112758,897.629150,425153.343338
59369,Taken (2008),217.852467,801.589156,345002.436241
